In [14]:
from typing import Annotated

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [15]:
from langchain.chat_models import init_chat_model

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os 
from dotenv import load_dotenv
load_dotenv()
api_key = api_key=os.environ.get("GEMINI_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # Use the correct model
    google_api_key=api_key 
)

response = llm.invoke("What is the capital of Italy?")
print(response.content)


E0000 00:00:1762619540.816554 1206693 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


The capital of Italy is **Rome**.


In [18]:
# llm = init_chat_model("google_genai:gemini-2.0-flash")
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",  # Use the correct model
    google_api_key=api_key 
)

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

E0000 00:00:1762619603.967646 1206693 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [19]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='113798ea-dd1b-4701-9d56-3686e75efab6'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--ba2dc039-37c0-4ee5-b7cf-a50050cdb18a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 3, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [20]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: The current Prime Minister of India is Narendra Modi.
Bot: Narendra Modi was born on September 17, 1950. As of today, November 2, 2023, he is 73 years old.
Bot: It's nice to meet you, Prakash! How can I help you today?
Bot: Your name is Prakash. You just told me that!
Bot: You're very welcome, Prakash! I'm glad I could help. Thank you for the kind words! Is there anything else I can do for you today?
